In [90]:
import tpqoa

In [99]:
api = tpqoa.tpqoa("/Users/dugaldmacintyre/Desktop/Oanda_firststeps/oanda.cfg")

import sys
print(sys.executable)

/Users/dugaldmacintyre/anaconda3/bin/python


In [92]:
help(api.get_history)

Help on method get_history in module tpqoa.tpqoa:

get_history(instrument, start, end, granularity, price, localize=True) method of tpqoa.tpqoa.tpqoa instance
    Retrieves historical data for instrument.
    
    Parameters
    instrument: string
        valid instrument name
    start, end: datetime, str
        Python datetime or string objects for start and end
    granularity: string
        a string like 'S5', 'M1' or 'D'
    price: string
        one of 'A' (ask), 'B' (bid) or 'M' (middle)
    
    Returns
    data: pd.DataFrame
        pandas DataFrame object with data



In [93]:
api.get_history(instrument="EUR_USD", start="2020-07-01", end="2020-07-31", granularity="D", price="B")


,o,h,l,c,volume,complete
time,,,,,,
2020-06-30 21:00:00,1.12293,1.12744,1.11837,1.12501,90252,True
2020-07-01 21:00:00,1.12501,1.13020,1.12188,1.12384,90789,True
2020-07-02 21:00:00,1.12384,1.12492,1.12186,1.12429,59036,True
2020-07-05 21:00:00,1.12423,1.13449,1.12391,1.13076,81756,True
2020-07-06 21:00:00,1.13070,1.13318,1.12581,1.12726,92426,True
2020-07-07 21:00:00,1.12717,1.13505,1.12615,1.13287,89549,True
2020-07-08 21:00:00,1.13287,1.13699,1.12774,1.12837,104729,True
2020-07-09 21:00:00,1.12808,1.13242,1.12540,1.12977,81510,True
2020-07-12 21:00:00,1.13042,1.13744,1.13002,1.13420,86512,True


In [101]:
api.get_history(instrument="EUR_USD", start="2020-07-01", end="2020-07-31", granularity="M5", price="A")[["o", "c"]]

,o,c
time,,
2020-07-01 00:00:00,1.12344,1.12302
2020-07-01 00:05:00,1.12300,1.12298
2020-07-01 00:10:00,1.12300,1.12313
2020-07-01 00:15:00,1.12315,1.12319
2020-07-01 00:20:00,1.12316,1.12302
...,...,...
2020-07-30 23:35:00,1.18651,1.18665
2020-07-30 23:40:00,1.18663,1.18676
2020-07-30 23:45:00,1.18679,1.18674


In [102]:
# Fetching ask and bid prices
ask_data = api.get_history(instrument="EUR_USD", start="2020-07-01", end="2020-07-31", granularity="M5", price="A")
bid_data = api.get_history(instrument="EUR_USD", start="2020-07-01", end="2020-07-31", granularity="M5", price="B")

# Creating a DataFrame for ask and bid prices
ask_close = ask_data[["c"]]
bid_close = bid_data[["c"]]

# Calculating price as the average of ask and bid
df = pd.DataFrame()
df["price"] = (ask_close + bid_close) / 2

# Adding close prices, bid-ask spread, and renaming columns
df["spread"] = ask_close.sub(bid_close)

# Display the resulting DataFrame
df

,price,spread
time,,
2020-07-01 00:00:00,1.122940,0.00016
2020-07-01 00:05:00,1.122905,0.00015
2020-07-01 00:10:00,1.123055,0.00015
2020-07-01 00:15:00,1.123105,0.00017
2020-07-01 00:20:00,1.122925,0.00019
...,...,...
2020-07-30 23:35:00,1.186570,0.00016
2020-07-30 23:40:00,1.186670,0.00018
2020-07-30 23:45:00,1.186655,0.00017


In [96]:
df["close_ask"] = api.get_history(instrument="EUR_USD", start="2020-07-01", end="2020-07-31", granularity="D", price="A")[["c"]]
df["close_bid"] = api.get_history(instrument="EUR_USD", start="2020-07-01", end="2020-07-31", granularity="D", price="B")[["c"]]
df["spread"] = df.close_ask.sub(df.close_bid)
df = df[["c", "spread"]]
df.rename(columns={"c": "price"})

,price,spread
time,,
2020-06-30 21:00:00,1.12527,0.00026
2020-07-01 21:00:00,1.12403,0.00019
2020-07-02 21:00:00,1.12529,0.00100
2020-07-05 21:00:00,1.13113,0.00037
2020-07-06 21:00:00,1.12762,0.00036
2020-07-07 21:00:00,1.13310,0.00023
2020-07-08 21:00:00,1.12868,0.00031
2020-07-09 21:00:00,1.13028,0.00051
2020-07-12 21:00:00,1.13443,0.00023


In [ ]:

from IterativeBase import *

class IterativeBacktest(IterativeBase):
    ''' Class for iterative (event-driven) backtesting of trading strategies.
    '''

    # helper method
    def go_long(self, bar, units = None, amount = None):
        if self.position == -1:
            self.buy_instrument(bar, units = -self.units) # if short position, go neutral first
        if units:
            self.buy_instrument(bar, units = units)
        elif amount:
            if amount == "all":
                amount = self.current_balance
            self.buy_instrument(bar, amount = amount) # go long

    # helper method
    def go_short(self, bar, units = None, amount = None):
        if self.position == 1:
            self.sell_instrument(bar, units = self.units) # if long position, go neutral first
        if units:
            self.sell_instrument(bar, units = units)
        elif amount:
            if amount == "all":
                amount = self.current_balance
            self.sell_instrument(bar, amount = amount) # go short

    def test_sma_strategy(self, SMA_S, SMA_L):
        
        # nice printout
        stm = "Testing SMA strategy | {} | SMA_S = {} & SMA_L = {}".format(self.symbol, SMA_S, SMA_L)
        print("-" * 75)
        print(stm)
        print("-" * 75)
        
        # reset 
        self.position = 0  # initial neutral position
        self.trades = 0  # no trades yet
        self.current_balance = self.initial_balance  # reset initial capital
        self.get_data() # reset dataset
        
        # prepare data
        self.data["SMA_S"] = self.data["price"].rolling(SMA_S).mean()
        self.data["SMA_L"] = self.data["price"].rolling(SMA_L).mean()
        self.data.dropna(inplace = True)

        # sma crossover strategy
        for bar in range(len(self.data)-1): # all bars (except the last bar)
            if self.data["SMA_S"].iloc[bar] > self.data["SMA_L"].iloc[bar]: # signal to go long
                if self.position in [0, -1]:
                    self.go_long(bar, amount = "all") # go long with full amount
                    self.position = 1  # long position
            elif self.data["SMA_S"].iloc[bar] < self.data["SMA_L"].iloc[bar]: # signal to go short
                if self.position in [0, 1]:
                    self.go_short(bar, amount = "all") # go short with full amount
                    self.position = -1 # short position
        self.close_pos(bar+1) # close position at the last bar
        
        
    def test_con_strategy(self, window = 1):
        
        # nice printout
        stm = "Testing Contrarian strategy | {} | Window = {}".format(self.symbol, window)
        print("-" * 75)
        print(stm)
        print("-" * 75)
        
        # reset 
        self.position = 0  # initial neutral position
        self.trades = 0  # no trades yet
        self.current_balance = self.initial_balance  # reset initial capital
        self.get_data() # reset dataset
        
        # prepare data
        self.data["rolling_returns"] = self.data["returns"].rolling(window).mean()
        self.data.dropna(inplace = True)
        
        # Contrarian strategy
        for bar in range(len(self.data)-1): # all bars (except the last bar)
            if self.data["rolling_returns"].iloc[bar] <= 0: #signal to go long
                if self.position in [0, -1]:
                    self.go_long(bar, amount = "all") # go long with full amount
                    self.position = 1  # long position
            elif self.data["rolling_returns"].iloc[bar] > 0: #signal to go short
                if self.position in [0, 1]:
                    self.go_short(bar, amount = "all") # go short with full amount
                    self.position = -1 # short position
        self.close_pos(bar+1) # close position at the last bar
        
        
    def test_boll_strategy(self, SMA, dev):

        # nice printout
        stm = "Testing Bollinger Bands Strategy | {} | SMA = {} & dev = {}".format(self.symbol, SMA, dev)
        print("-" * 75)
        print(stm)
        print("-" * 75)
        
        # reset 
        self.position = 0  # initial neutral position
        self.trades = 0  # no trades yet
        self.current_balance = self.initial_balance  # reset initial capital
        self.get_data() # reset dataset
        
        # prepare data
        self.data["SMA"] = self.data["price"].rolling(SMA).mean()
        self.data["Lower"] = self.data["SMA"] - self.data["price"].rolling(SMA).std() * dev
        self.data["Upper"] = self.data["SMA"] + self.data["price"].rolling(SMA).std() * dev
        self.data.dropna(inplace = True) 
        
        # Bollinger strategy
        for bar in range(len(self.data)-1): # all bars (except the last bar)
            if self.position == 0: # when neutral
                if self.data["price"].iloc[bar] < self.data["Lower"].iloc[bar]: # signal to go long
                    self.go_long(bar, amount = "all") # go long with full amount
                    self.position = 1  # long position
                elif self.data["price"].iloc[bar] > self.data["Upper"].iloc[bar]: # signal to go Short
                    self.go_short(bar, amount = "all") # go short with full amount
                    self.position = -1 # short position
            elif self.position == 1: # when long
                if self.data["price"].iloc[bar] > self.data["SMA"].iloc[bar]:
                    if self.data["price"].iloc[bar] > self.data["Upper"].iloc[bar]: # signal to go short
                        self.go_short(bar, amount = "all") # go short with full amount
                        self.position = -1 # short position
                    else:
                        self.sell_instrument(bar, units = self.units) # go neutral
                        self.position = 0
            elif self.position == -1: # when short
                if self.data["price"].iloc[bar] < self.data["SMA"].iloc[bar]:
                    if self.data["price"].iloc[bar] < self.data["Lower"].iloc[bar]: # signal to go long
                        self.go_long(bar, amount = "all") # go long with full amount
                        self.position = 1 # long position
                    else:
                        self.buy_instrument(bar, units = -self.units) # go neutral
                        self.position = 0                
        self.close_pos(bar+1) # close position at the last bar